# Buchhandelsdaten in Vufind als Grundlage für PDA (Patron Driven Aquisition) am MPIfG / Version 2: Updates laden statt Vollimport

Einbindung von freundlicherweise von Schweitzer Fachinformation zur Verfügung gestellten Daten.   
(Auswahl über passend konfigurierte Neuerscheinungsabfragen in unserem Kundenprofil).   

#### Gründe für die Umstellung auf Updates: Durch Einspielen der Daten in Vufind ergibt sich eine zeitliche Diskrepanz, in der Titel bereits in Aleph gelöscht sind, aber in Vufind noch bestellbar. Zudem werden die Aleph-IDs hochgezähl, denn bei jedem Einspielen werden neue Nummern vergeben. Durch die Updates bleiben die Titel weiterhin verfügbar und das Hochzählen der IDs hält sich im Rahmen. Das Skript beschleunigt sich durch Reduzierung der Abfragen am Server.
*Kleiner Nachteil: der manuelle Aufwand vergrößert sich. Mehrere Dateien müssen in Aleph eingespielt und verarbeitet werden, aber hält sich nach wie vor im vertretbaren Rahmen.*

Das Jupyter Notebook arbeitet mit Python 3.8.1 und wurde mit Visual Studio Code 1.62.3 erstellt 


#### Arbeitsschritte im Code:

> Vorarbeiten:   
  - Notwendige Pandas Libraries aufrufen
  - Serverprüfung auf funktionierende Verbindung zum Aleph-X-Server    


> Daten abholen und einlesen:   
  1. Buchhandelsdaten von Schweitzer 
  2. Aleph-Konkordanz Aleph-ID /Schweitzer ID  (erzeugt tagesaktuell per p-print-03 in Aleph)
     - Aufbereiten der Daten: Schweitzer ID extrahieren und Aleph-ID mit Nullen auffüllen
  3. Daten zusammenführen in einem df    


> Daten aufbereiten:   
  1. Buchhandelsdaten prüfen und vorbereiten
     1. Identifizierung von Titeln in Aleph, die nicht mehr im Datensatz sind und schreiben in Datei "ges02_loeschen_1"
     2. Dublettencheck innerhalb der Buchhandelsdaten
     3. Trennung der Daten in "in Aleph vorhanden" und "neu"  
  2. Bestandsabfragen:
     1. Ganz neue Titel   
        - Bestandsabgleich durch Abfragen (GES und EBX) auf dem Aleph-Server   
     2. In Aleph vorhanden
        - Bestandsabgleich durch Abfrage EBX auf dem Aleph-Server  
     3. Exportvorbereitungen:
        1. Neue Titel
        2. In Aleph vorhandene Titel   
            Identifizierung von zu löschenden Titeln "ges02_loeschen_2" und der zu aktualisierenden
  3. Exportdateien aufbereiten:
     1. Aufbereiten neuer Titel
     2. Aufbereiten der vorhandenen Titel   


> Informationssammlung
   1. Log-Datei mit Rahmendaten wird fortlaufend geschrieben
   2. Ausgabe bestimmter Titelgruppen als csv-Datei  


<hr>

## Vorarbeiten

### Pandas Libraries laden

In [1]:
import pandas as pd                                                # für das Arbeiten mit der CSV-Datei
import urllib.request                                              # für das Abrufen der URL
import requests                                                    # für die Bestandsabfragen 
pd.options.mode.chained_assignment = None                          # default='warn' abschalten beim Beschreiben der neuen Spalten
import time                                                        # für das Schreiben des Datums Logdatei und Excel-Export und Arbeiten mit dem Erscheinungsdatum
import datetime                                                    # für das Berechnen des Updates
import numpy as np                                                 # für das Bearbeiten von Spalten
from datetime import datetime                                      # für das Laden des Datums
from datetime import datetime, timedelta                           # für die Berechnung des letzten Monats

### Prüfung, ob die Verbindung zum Aleph-Server für Abfragen korrekt funktioniert:

    Nur zugelassene IPs können diese Schnittstelle abfragen.

In [2]:
test= "http://aleph.mpg.de/X?op=find&base=ges01&request=IBS=9783482648434"

reply = requests.get(test).text
a = reply.find("Forbidden")
b =  reply.find("?xml")

if (a > 50):
    print("Es gibt ein Problem mit dem Server")
if (b == 1):
    print("Der Server antwortet korrekt")

Der Server antwortet korrekt


In [3]:
test= "http://aleph.mpg.de/X?op=find&base=ges01&request=IBS=9783865058041"

reply = requests.get(test).text 

print(reply) 

<?xml version = "1.0" encoding = "UTF-8"?>
<find>
<set_number>043032</set_number>
<no_records>000000001</no_records>
<no_entries>000000001</no_entries>
<session-id>GJJD43G3BFJI64RIPM4G84D1MQ237117M2AP49Y31IK33PST2M</session-id>
</find>
 


<hr>


## Datensätze abholen und einlesen

# Erste Schritte manuell durchführen, um enthaltene Prüfroutinen im Blick zu behalten

<hr>

### 1. Datensätze von Schweitzer

*einlesen in df*

In [4]:
url = "https://content.schweitzer-online.de/static/content/export/mpifg/export.csv"  # Abruf, der von Schweitzer zur Verfügung gestellten Daten
checkout_file = "./input/export.csv"  
urllib.request.urlretrieve(url, checkout_file)

('./input/export.csv', <http.client.HTTPMessage at 0x205e79fd340>)

In [5]:
df = pd.read_csv('./input/export.csv', encoding = 'UTF-8', sep=';' , keep_default_na=False , low_memory=False) # muss encoding angeben und Trennzeichen, NaN (= leere Werte) direkt beim Import entfernen, da sie später Probleme machen

In [6]:
#Aufgrund eines zusätzlichen Zeilenumbruchs im Datensatz gab es im Juni 2022 ein Problem, darum kleine Prüfroutine eingebaut ob in der Import-Datei die Anzahl der Datensätze mit der Anzahl der object-Ids übereinstimmt
g = df.shape[0]
h= df['object_id'].count()

print("Zeilen in Datei:", g)
print("Object_IDs:     ", h)


Zeilen in Datei: 66994
Object_IDs:      66994


In [7]:
#df.info()

In [8]:
df

,object_id,isbn_ean,title,subtitle,contributor,publisher,series,thesis,edition_number,edition_text,...,publication_date,delivery_signal,delivery_text,price,description,cover,predecessor,follower,last_modified,watchlist_name
0,2258402,9783848723201,Europäisches Arbeitsrecht,,Dagmar Schiek;Rebecca Zahn,Nomos,,,4,,...,20250307,green,delivery_period:available_immediately,4990,Arbeitsrechtliche Beziehungen werden sehr star...,https://content.schweitzer-online.de/static/ca...,"9783832930561,9783832912536,9783789050626",,2025-03-23 21:03:31,Himbeere Wirtschaft #34
1,2522807,9783848768073,Klimaschutzrecht,Bundes-Klimaschutzgesetz | Landesklimagesetze ...,Charlotte Kreuter-Kirchhof,Nomos,NomosHandkommentar,,1,,...,20250500,yellow,delivery_status:10,14900,Das neue Recht Mit dem Bundes-Klimaschutzgeset...,https://content.schweitzer-online.de/static/ca...,,,2025-02-07 06:11:51,brombeere Politik #139
2,2308946,9783848728497,Rehabilitation und Teilhabe,,,Nomos,Kompendien der Sozialen Arbeit Band 5,,1,,...,20250700,yellow,delivery_status:10,2490,Das Kompendium bietet Studierenden und Fachkrä...,https://content.schweitzer-online.de/static/ca...,,,2025-03-17 20:12:18,Himbeere Wirtschaft #168
3,2220214,9783406701948,Besteuerung privater Kapitalanlagen,"Finanzinstrumente, Investmentanteile, Immobili...",Hans-Jürgen A. Feyerabend,C.H.BECK,,,2,,...,20250831,yellow,delivery_status:10,14900,Dieses Handbuch bietet eine umfassende Darstel...,https://content.schweitzer-online.de/static/ca...,9783406575297,,2025-02-20 20:40:50,brombeere Politik #78.1
4,2489418,9783825254476,Biodiversität? Frag doch einfach!,Klare Antworten aus erster Hand,Heike Feldhaar,UTB,Frag doch einfach!,,1,,...,20250512,yellow,delivery_status:10,1990,Durch den Klimawandel gerät auch die Biodivers...,https://content.schweitzer-online.de/static/ca...,,,2025-02-18 06:40:09,brombeere Politik #165.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66989,173144408,9782336518268,La politique de l'emploi au Sénégal,"Cinq questions pour comprendre, agir et impacter",Demba Diop,Harmattan Sénégal,,,,,...,20250314,green,delivery_period:available_3to4weeks,2200,L'emploi est au centre des discours et préoccu...,,,,2025-03-21 05:59:49,Himbeere Wirtschaft #201
66990,173144418,9782336523149,Emploi et travail au Sénégal,,Alioune Kébé,Harmattan Sénégal,,,,,...,20250314,green,delivery_period:available_3to4weeks,1400,Cet ouvrage se propose de réconcilier les aspi...,,,,2025-03-21 05:59:50,Himbeere Wirtschaft #201
66991,173144396,9782336496757,À la recherche d'un Tchad meilleur ?,Bâtir une nation forte pour un avenir de dével...,Beckri Hammid Haggar Djougourou,Harmattan Sénégal,,,,,...,20250314,green,delivery_period:available_3to4weeks,1500,Plongez dès maintenant dans cet ouvrage percut...,,,,2025-03-21 05:59:49,Himbeere Wirtschaft #201
66992,173144409,9782336518350,Macroéconomie,Théories et applications de politiques économi...,Youssouf Keita,Editions L'Harmattan,,,,,...,20250314,green,delivery_period:available_3to4weeks,3100,"Loin d'être un débat doctrinaire, ce livre cou...",,,,2025-03-21 05:59:49,Himbeere Wirtschaft #201


# Ab dem nächsten Schritt kann Skript durchlaufen, Prüfroutinen sind erfolgt

<hr>

### 2. Aleph-Konkordanz Aleph-ID / Schweitzer ID

In [9]:
now = time.strftime('%Y%m%d')  # %H:%M:%S
input_file = './input/ids'+now      #input_file wird tagesaktuell aus Aleph gezogen und auf diesem Wege mit der entsprechenden Endung eingelesen

In [10]:
# neu einlesen der Aleph-IDs mittels Code, da sonst Object-Ids abgeschnitten wurden
with open(input_file) as f:
    data = f.readlines()

df2 = pd.DataFrame(data[0:]) # 0 weil ich keinen header habe

# use expand to split strings into separate columns
df_alephIDs = df2[0].str.rsplit(expand=True)
# fix column names
df_alephIDs.columns = ["ids","Field","L","Content"]

df_alephIDs['LEN'] = df_alephIDs['Content'].apply(len)
print(df_alephIDs)

             ids Field  L       Content  LEN
0      000129218   020  L    $$a2258402   10
1      000129219   020  L    $$a2522807   10
2      000129220   020  L    $$a2308946   10
3      000129221   020  L    $$a2220214   10
4      000129222   020  L    $$a2479903   10
...          ...   ... ..           ...  ...
29967  000159265   020  L  $$a169951393   12
29968  000159266   020  L  $$a163496075   12
29969  000159267   020  L  $$a165662822   12
29970  000159268   020  L  $$a170244345   12
29971  000159269   020  L  $$a171634215   12

[29972 rows x 5 columns]


*Datenfelder aufbereiten: Schweitzer-ID extrahieren und Aleph-ID ins richtige Format bringen*

In [11]:
df_alephIDs["object_id"] = df_alephIDs["Content"].astype(str).str.slice(start=3,stop=16).apply(int)   #Spalte mit object-Ids herausschneiden und wieder zur Zahl definieren

df_alephIDs["object_id"].value_counts() #Hier muss immer eins Stehen in der hinteren Spalte

2258402      1
170375246    1
170328426    1
170384685    1
170346810    1
            ..
167647450    1
167647389    1
167645502    1
167645335    1
171634215    1
Name: object_id, Length: 29972, dtype: int64

### 3. Daten zu einem Frame zusammenführen

In [12]:
# Zusammenführung erfolgt einmal mit concat - erlaubt die nicht mehr verwendeten Aleph-IDs zu identifizieren 
# und einmal mit join, dann erhalte ich nur die Titel, die auch im neuen Datensatz sind zur weiteren Verarbeitung!
df_oi= df.set_index("object_id")                          #object-ID zum Index für beide Datenframes
df_aleph_oi = df_alephIDs.set_index("object_id")
df_update_aleph = pd.concat([df_oi, df_aleph_oi], axis=1)

df_update_join = df_oi.join(df_aleph_oi)                    



In [13]:
df_update_join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66994 entries, 2258402 to 173144412
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   isbn_ean          66994 non-null  int64  
 1   title             66994 non-null  object 
 2   subtitle          66994 non-null  object 
 3   contributor       66994 non-null  object 
 4   publisher         66994 non-null  object 
 5   series            66994 non-null  object 
 6   thesis            66994 non-null  object 
 7   edition_number    66994 non-null  object 
 8   edition_text      66994 non-null  object 
 9   media_type        66994 non-null  object 
 10  pages             66994 non-null  object 
 11  publication_year  66994 non-null  object 
 12  publication_date  66994 non-null  object 
 13  delivery_signal   66994 non-null  object 
 14  delivery_text     66994 non-null  object 
 15  price             66994 non-null  int64  
 16  description       66994 non-nu

<hr>
### Wenn es bei der obigen Zelle zu Fehlermeldung kommt, dann gibt es vermutlich doppelte SChweitzer-IDs in den Daten.

Das kann man beim Output der folgenden Zelle oben sehen:

    #nach Dubletten vom letzten Mal vorsichtshalber, check nach doppelten object-ids
    df_alephIDs["object_id"].value_counts()

Wenn da Nummern mit 2 vorhanden sind, dann muss oben der Code zur Dublettenbereinigung gestartet werden. Dies geschieht, in dem die folgende Zelle von Markdown zu Code geändert wird und durchlaufen. 

## Dabei wird eine weitere Datei erzeugt, die in Aleph gelöscht werden muss!!

WICHTIG: Anschließend, den Code wieder in Markdown verwandeln, da er ja nur in dem Sonderfall gebraucht wird. 

<hr>

# Code zur Bereinigung von Dubletten im in Aleph schon eingespielten Bestand 
# Einmalig aktivieren (Markdown > Code) und nutzen und dann wieder (Code > Markdown)

df_aleph_einzel = df_alephIDs.drop_duplicates("object_id", keep=False)  #Auslesen der Eintraege mit einzelner object_id
df_aleph_doppelt = df_alephIDs.groupby("object_id").filter(lambda g: (g.nunique() >1).any()) #Alle Sätze mit doppelten object_ids rausziehen
df_aleph_single = df_aleph_doppelt.sort_values(by=["object_id", "ids"], ascending =False).drop_duplicates(subset=["object_id"], keep='first') #Die Dublette, die behalten wird
df_aleph_dublette = df_aleph_doppelt.sort_values(by=["object_id", "ids"], ascending =False).drop_duplicates(subset=["object_id"], keep='last') #Die Dublette, die gelöscht wird
# In Aleph zu löschende in eine Datei schreiben aus df_aleph_dublette
with open("./output/ges02_dubl", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_aleph_dublette.index:
        fa.write(df_aleph_dublette["ids"][i]+'GES02'+'\n')
        
df_aleph_vorhanden = df_aleph_einzel.append(df_aleph_single) #Einzelne und ausgewählt Dubletten zusammenfuehren

df_oi= df.set_index("object_id")                          #Dann nochmal den Schritt, an dem es vorhin gehakt hat
df_aleph_oi = df_aleph_vorhanden.set_index("object_id")
df_update_aleph = pd.concat([df_oi, df_aleph_oi], axis=1)

df_update_join = df_oi.join(df_aleph_oi)   

# Noch kleine Rechnung zur Ueberprüfung, ob berechnete Dubletten und mit Skript ermittelte Dublettenzahl übereinstimmen
g = df_alephIDs.shape[0]
h= df_aleph_einzel.shape[0]
j = df_aleph_doppelt.shape[0]

print("Dubletten berechnet: ",  g-h)
print("Dubletten gezählt:   ", j)  


## Wenn alles gut ist, kann das Skript ab hier wieder mit "all Cells below" durchlaufen
<hr>

### LOG-Datei für den Prozess, zur Dokumentation des Imports und als Kontrollanzeige hier


In [14]:
x = df.shape[0]
print('Anzahl der enthaltenen Datensätze:', x)
#print('vorhandene ISBNs:', df_update["isbn_ean"].shape[0])

z = df_alephIDs["ids"].count()
y = df_update_join["ids"].count()

print('-------------------------')
print('Aleph-IDs anfangs', z)
print('Aleph-IDs nach join' , y)

timestr = time.strftime('%d.%m.%Y - %H:%M')

with open ('./log/pda_import_log.txt', 'a') as log:                                                  # Da diese Log-Datei nicht unmittelbar gebraucht wird, hier fortlaufendes Schreiben in eine Datei
    log.write('Logdatei PDA-Import vom ')
    log.write(timestr)
    log.write('\n------------------------------------------\n')
    log.write('Gelieferte Datensätze:             ' + str(x))

Anzahl der enthaltenen Datensätze: 66994
-------------------------
Aleph-IDs anfangs 29972
Aleph-IDs nach join 29762


## Daten aufbereiten

<hr>

### 1. Identifizierung von Titeln in Aleph-Dubletten, die nicht mehr im Datensatz sind

In [15]:
df_update_aleph.replace({np.nan: None})

,isbn_ean,title,subtitle,contributor,publisher,series,thesis,edition_number,edition_text,media_type,...,cover,predecessor,follower,last_modified,watchlist_name,ids,Field,L,Content,LEN
object_id,,,,,,,,,,,,,,,,,,,,,
2258402,9783848723201.0,Europäisches Arbeitsrecht,,Dagmar Schiek;Rebecca Zahn,Nomos,,,4,,hardcover,...,https://content.schweitzer-online.de/static/ca...,"9783832930561,9783832912536,9783789050626",,2025-03-23 21:03:31,Himbeere Wirtschaft #34,000129218,020,L,$$a2258402,10.0
2522807,9783848768073.0,Klimaschutzrecht,Bundes-Klimaschutzgesetz | Landesklimagesetze ...,Charlotte Kreuter-Kirchhof,Nomos,NomosHandkommentar,,1,,hardcover,...,https://content.schweitzer-online.de/static/ca...,,,2025-02-07 06:11:51,brombeere Politik #139,000129219,020,L,$$a2522807,10.0
2308946,9783848728497.0,Rehabilitation und Teilhabe,,,Nomos,Kompendien der Sozialen Arbeit Band 5,,1,,hardcover,...,https://content.schweitzer-online.de/static/ca...,,,2025-03-17 20:12:18,Himbeere Wirtschaft #168,000129220,020,L,$$a2308946,10.0
2220214,9783406701948.0,Besteuerung privater Kapitalanlagen,"Finanzinstrumente, Investmentanteile, Immobili...",Hans-Jürgen A. Feyerabend,C.H.BECK,,,2,,hardcover,...,https://content.schweitzer-online.de/static/ca...,9783406575297,,2025-02-20 20:40:50,brombeere Politik #78.1,000129221,020,L,$$a2220214,10.0
2489418,9783825254476.0,Biodiversität? Frag doch einfach!,Klare Antworten aus erster Hand,Heike Feldhaar,UTB,Frag doch einfach!,,1,,hardcover,...,https://content.schweitzer-online.de/static/ca...,,,2025-02-18 06:40:09,brombeere Politik #165.1,000129223,020,L,$$a2489418,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167722841,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,000156817,020,L,$$a167722841,12.0
170411317,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,000157031,020,L,$$a170411317,12.0
167545311,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,000157718,020,L,$$a167545311,12.0


In [16]:
df_aleph_loeschen = df_update_aleph[df_update_aleph['isbn_ean'].isna()]  


In [17]:
df_aleph_loeschen

,isbn_ean,title,subtitle,contributor,publisher,series,thesis,edition_number,edition_text,media_type,...,cover,predecessor,follower,last_modified,watchlist_name,ids,Field,L,Content,LEN
object_id,,,,,,,,,,,,,,,,,,,,,
2254200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,000129227,020,L,$$a2254200,10.0
2481913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,000129235,020,L,$$a2481913,10.0
2430847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,000129244,020,L,$$a2430847,10.0
2368410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,000129249,020,L,$$a2368410,10.0
2544500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,000129260,020,L,$$a2544500,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167722841,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,000156817,020,L,$$a167722841,12.0
170411317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,000157031,020,L,$$a170411317,12.0
167545311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,000157718,020,L,$$a167545311,12.0


In [18]:
# Schreiben der Loeschen-datei!

with open("./output/ges02_weg", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_aleph_loeschen.index:
        fa.write(df_aleph_loeschen["ids"][i]+'GES02'+'\n')

<hr>

## 2. Dublettencheck innerhalb gelieferten Buchhandelsdaten 

Aufgaben im Rahmen des Dublettencheck:
1. Dublettenkontrolle anhand von Titel, Untertitel und Autor 
   - Zunächst Behebung der unsauberen Titel / Untertitel-Trennung für korrekteren Abgleich
   - Trennung der Datensätze in Dubletten und "Einzeltitel"
     - Einzeltitel werden direkt für Bestandsprüfung vorgemerkt
     - Dubletten werden auf neueste Version reduziert und diese der Bestandsprüfungsdatei hinzugefügt

### Die Dublettenkontrolle Anhand von Titel, Untertitel und Autor

*Entfernen von Untertiteln aus der Titelspalte, Extrahieren von Untertiteln und Abgleich mit Untertitelspalte und Schreiben der vorhandenen Informationen in neue Untertitel-Spalte.   
Durch diese Spalte werden ca. 1/3 mehr Dubletten erkannt, als ohne die Bereinigung. *

In [19]:
df = df_update_join[df_update_join['isbn_ean'].notna()]  #jetzt alle die zu verarbeiten sind rausziehen
df.reset_index(inplace=True) 

In [20]:
neu = df["title"].str.split(':', n = 1, expand = True)  #Titel am 1. Doppelpunkt splitten und getrennt in neue Felder schreiben
df["title_sep"]= neu[0]
df["subtitle_sep"]= neu[1]

df["subtitle_sep"] = df["subtitle_sep"].replace(np.nan, '', regex=True) #NaN-Werte stören, darum raus damit ...

In [21]:
comparison = np.where(df["subtitle"] == df["subtitle_sep"], '', df["subtitle"])    # Abgleich - wenn in beiden das Gleiche steht, dann ursprüngliches "Subtitle"-Feld nehmen
df["subtitle_comparison"] = comparison  

comparison2 = np.where(df["subtitle"] < df["subtitle_sep"], df["subtitle_sep"], '') # Wenn nur in "subtitle_sep" Infos stehen, diese übernehmen, das ist noch nicht ganz sauber, da hier manchmal anderes steht als in "subtitle"
df["subtitle_comparison2"] = comparison2 

df["subtitle_all"] = df["subtitle_comparison"]+df["subtitle_comparison2"]          # Beide Informationen in neuer Subtitle-Spalte zusammenführen

In [22]:
df["short_title"] = df["title_sep"] + ' ' + df["subtitle_all"] + ' / ' + df["contributor"]  # aus den bereinigten Daten einen Kurztitel erzeugen, der dann für den Dublettencheck verwendet wird

In [23]:
df_dubletten = df.groupby("short_title").filter(lambda g: (g.nunique() >1).any()) # schreibt alle mehrfach vorhandenen Titel in ein eigenes Datenframe
df_dubletten

,object_id,isbn_ean,title,subtitle,contributor,publisher,series,thesis,edition_number,edition_text,...,Field,L,Content,LEN,title_sep,subtitle_sep,subtitle_comparison,subtitle_comparison2,subtitle_all,short_title
27,2382420,9783406723865,Die weltweite Ungleichheit,Der World Inequality Report 2018,Facundo Alvaredo,C.H.BECK,Beck Paperback,,1,1. Auflage,...,NaN,NaN,NaN,NaN,Die weltweite Ungleichheit,,Der World Inequality Report 2018,,Der World Inequality Report 2018,Die weltweite Ungleichheit Der World Inequalit...
30,2495796,9783898798822,Rich Dad Poor Dad,Was die Reichen ihren Kindern über Geld beibri...,Robert T. Kiyosaki,FinanzBuch Verlag,,,12,,...,NaN,NaN,NaN,NaN,Rich Dad Poor Dad,,Was die Reichen ihren Kindern über Geld beibri...,,Was die Reichen ihren Kindern über Geld beibri...,Rich Dad Poor Dad Was die Reichen ihren Kinder...
44,2382419,9783406723858,Die weltweite Ungleichheit,Der World Inequality Report 2018,Facundo Alvaredo,C.H.BECK,,,1,,...,NaN,NaN,NaN,NaN,Die weltweite Ungleichheit,,Der World Inequality Report 2018,,Der World Inequality Report 2018,Die weltweite Ungleichheit Der World Inequalit...
54,2458299,9780745651125,Intimacy,Personal Relationships in Modern Societies,Lynn Jamieson,Polity Press,,,2,,...,NaN,NaN,NaN,NaN,Intimacy,,Personal Relationships in Modern Societies,,Personal Relationships in Modern Societies,Intimacy Personal Relationships in Modern Soci...
55,2458300,9780745651132,Intimacy,Personal Relationships in Modern Societies,Lynn Jamieson,Polity Press,,,2,,...,NaN,NaN,NaN,NaN,Intimacy,,Personal Relationships in Modern Societies,,Personal Relationships in Modern Societies,Intimacy Personal Relationships in Modern Soci...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66963,173106774,9781009324243,The Metaphysics of Color,,Michael Watkins;Elay Shech,Cambridge University Press,Elements in Metaphysics,,,,...,NaN,NaN,NaN,NaN,The Metaphysics of Color,,,,,The Metaphysics of Color / Michael Watkins;El...
66964,173150253,9781009068468,Rethinking Colonial Legacies across Southeast ...,Through the Lens of the Japanese Wartime Empire,Diana S. Kim,Cambridge University Press,Elements in Politics and Society in Southeast ...,,,,...,NaN,NaN,NaN,NaN,Rethinking Colonial Legacies across Southeast ...,,Through the Lens of the Japanese Wartime Empire,,Through the Lens of the Japanese Wartime Empire,Rethinking Colonial Legacies across Southeast ...
66965,173150292,9781009671484,Rethinking Colonial Legacies across Southeast ...,Through the Lens of the Japanese Wartime Empire,Diana S. Kim,Cambridge University Press,Elements in Politics and Society in Southeast ...,,,,...,NaN,NaN,NaN,NaN,Rethinking Colonial Legacies across Southeast ...,,Through the Lens of the Japanese Wartime Empire,,Through the Lens of the Japanese Wartime Empire,Rethinking Colonial Legacies across Southeast ...
66971,173157575,9780197802007,Workplace Politics,How Politicians and Employers Subvert Elections,Timothy M. Frye;Ora John Reuter;David Szakonyi,Oxford University Press,,,,,...,NaN,NaN,NaN,NaN,Workplace Politics,,How Politicians and Employers Subvert Elections,,How Politicians and Employers Subvert Elections,Workplace Politics How Politicians and Employe...


In [24]:
df_dubl_einspielen = df_dubletten.sort_values(by=["short_title", "publication_year"], ascending =False).drop_duplicates(subset=["short_title"], keep='first')   # sortiert Dubletten nach Jahr und schreibt den jeweils ersten (= neuesten) Eintrag in neues Dataframe
df_dubl_einspielen

,object_id,isbn_ean,title,subtitle,contributor,publisher,series,thesis,edition_number,edition_text,...,Field,L,Content,LEN,title_sep,subtitle_sep,subtitle_comparison,subtitle_comparison2,subtitle_all,short_title
66920,173056902,9783825264109,"Ökonomie, Moral und Geschichte",Eine themenorientierte Einführung,Alfons J. Weichenrieder,UTB,,,2,2. verb. u. aktual. Aufl.,...,NaN,NaN,NaN,NaN,"Ökonomie, Moral und Geschichte",,Eine themenorientierte Einführung,,Eine themenorientierte Einführung,"Ökonomie, Moral und Geschichte Eine themenorie..."
41868,167973599,9781478025672,dear elia,Letters from the Asian American Abyss,Mimi Khuc,Duke University Press,,,,,...,020,L,$$a167973599,12.0,dear elia,,Letters from the Asian American Abyss,,Letters from the Asian American Abyss,dear elia Letters from the Asian American Abys...
28937,164814177,9781642599022,ballast,,Quenton Baker,Haymarket Books,,,,,...,020,L,$$a164814177,12.0,ballast,,,,,ballast / Quenton Baker
22456,163769028,9780228013969,Zygmunt Bauman and the Theory of Culture,,Dariusz Brzezinski,McGill-Queen's University Press,,,,,...,NaN,NaN,NaN,NaN,Zygmunt Bauman and the Theory of Culture,,,,,Zygmunt Bauman and the Theory of Culture / Da...
34860,166346661,9781108468725,Zoo Studies,"Living Collections, Their Animals and Visitors",Paul A. Rees,Cambridge University Press,,,,,...,020,L,$$a166346661,12.0,Zoo Studies,,"Living Collections, Their Animals and Visitors",,"Living Collections, Their Animals and Visitors","Zoo Studies Living Collections, Their Animals ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20308,163496075,9783030962708,'Going Native?',Settler Colonialism and Food,Ronald Ranta,Palgrave Macmillan,Food and Identity in a Globalising World,,,2022 ed.,...,020,L,$$a163496075,12.0,'Going Native?',,Settler Colonialism and Food,,Settler Colonialism and Food,'Going Native?' Settler Colonialism and Food /...
32200,165662822,9781350276765,'Economy' in European History,"Words, Contexts and Change over Time",Luigi Alonzi,Bloomsbury Academic,,,,,...,020,L,$$a165662822,12.0,'Economy' in European History,,"Words, Contexts and Change over Time",,"Words, Contexts and Change over Time","'Economy' in European History Words, Contexts ..."
66009,172338230,9781009523103,#WakingTheFeminists and the Data-Driven Revolu...,,Claire Keogh,Cambridge University Press,Elements in Women Theatre Makers,,,,...,NaN,NaN,NaN,NaN,#WakingTheFeminists and the Data-Driven Revolu...,,,,,#WakingTheFeminists and the Data-Driven Revolu...
56987,170244345,9781538180648,#Gender,An Introduction to Gendered Social Problems,Sarah DeWard,Rowman & Littlefield,,,2,Second Edition,...,020,L,$$a170244345,12.0,#Gender,,An Introduction to Gendered Social Problems,,An Introduction to Gendered Social Problems,#Gender An Introduction to Gendered Social Pro...


In [25]:
df_ohne_dubletten = df.drop_duplicates("short_title", keep=False)       #durch ""keep=False" werden alle nicht-Dubletten rausgezogen
df_ohne_dubletten

,object_id,isbn_ean,title,subtitle,contributor,publisher,series,thesis,edition_number,edition_text,...,Field,L,Content,LEN,title_sep,subtitle_sep,subtitle_comparison,subtitle_comparison2,subtitle_all,short_title
0,2258402,9783848723201,Europäisches Arbeitsrecht,,Dagmar Schiek;Rebecca Zahn,Nomos,,,4,,...,020,L,$$a2258402,10.0,Europäisches Arbeitsrecht,,,,,Europäisches Arbeitsrecht / Dagmar Schiek;Reb...
1,2522807,9783848768073,Klimaschutzrecht,Bundes-Klimaschutzgesetz | Landesklimagesetze ...,Charlotte Kreuter-Kirchhof,Nomos,NomosHandkommentar,,1,,...,020,L,$$a2522807,10.0,Klimaschutzrecht,,Bundes-Klimaschutzgesetz | Landesklimagesetze ...,,Bundes-Klimaschutzgesetz | Landesklimagesetze ...,Klimaschutzrecht Bundes-Klimaschutzgesetz | La...
2,2308946,9783848728497,Rehabilitation und Teilhabe,,,Nomos,Kompendien der Sozialen Arbeit Band 5,,1,,...,020,L,$$a2308946,10.0,Rehabilitation und Teilhabe,,,,,Rehabilitation und Teilhabe /
3,2220214,9783406701948,Besteuerung privater Kapitalanlagen,"Finanzinstrumente, Investmentanteile, Immobili...",Hans-Jürgen A. Feyerabend,C.H.BECK,,,2,,...,020,L,$$a2220214,10.0,Besteuerung privater Kapitalanlagen,,"Finanzinstrumente, Investmentanteile, Immobili...",,"Finanzinstrumente, Investmentanteile, Immobili...",Besteuerung privater Kapitalanlagen Finanzinst...
4,2489418,9783825254476,Biodiversität? Frag doch einfach!,Klare Antworten aus erster Hand,Heike Feldhaar,UTB,Frag doch einfach!,,1,,...,020,L,$$a2489418,10.0,Biodiversität? Frag doch einfach!,,Klare Antworten aus erster Hand,,Klare Antworten aus erster Hand,Biodiversität? Frag doch einfach! Klare Antwor...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66989,173144408,9782336518268,La politique de l'emploi au Sénégal,"Cinq questions pour comprendre, agir et impacter",Demba Diop,Harmattan Sénégal,,,,,...,NaN,NaN,NaN,NaN,La politique de l'emploi au Sénégal,,"Cinq questions pour comprendre, agir et impacter",,"Cinq questions pour comprendre, agir et impacter",La politique de l'emploi au Sénégal Cinq quest...
66990,173144418,9782336523149,Emploi et travail au Sénégal,,Alioune Kébé,Harmattan Sénégal,,,,,...,NaN,NaN,NaN,NaN,Emploi et travail au Sénégal,,,,,Emploi et travail au Sénégal / Alioune Kébé
66991,173144396,9782336496757,À la recherche d'un Tchad meilleur ?,Bâtir une nation forte pour un avenir de dével...,Beckri Hammid Haggar Djougourou,Harmattan Sénégal,,,,,...,NaN,NaN,NaN,NaN,À la recherche d'un Tchad meilleur ?,,Bâtir une nation forte pour un avenir de dével...,,Bâtir une nation forte pour un avenir de dével...,À la recherche d'un Tchad meilleur ? Bâtir une...
66992,173144409,9782336518350,Macroéconomie,Théories et applications de politiques économi...,Youssouf Keita,Editions L'Harmattan,,,,,...,NaN,NaN,NaN,NaN,Macroéconomie,,Théories et applications de politiques économi...,,Théories et applications de politiques économi...,Macroéconomie Théories et applications de poli...


In [26]:
df_dubl_nicht_einspielen = df_dubletten.sort_values(by=["short_title", "publication_year"], ascending =False).drop_duplicates(subset=["short_title"], keep=False)

#Kontrolle, ob es in Dubletten Titel gibt, die schon in Aleph sind, wenn 0 ist gut
df_dubl_nicht_einspielen["ids"].count()


0

In [27]:
df_einspielen = df_ohne_dubletten.append(df_dubl_einspielen)                    # die ausgewählten Dubletten und alle Nicht-Dubletten werden in ein Datenframe zusammengeführt
df_einspielen.reset_index(inplace=True)                                         # für weitere Bearbeitung index-reset nötig

C:\Users\cm\AppData\Local\Temp\ipykernel_3328\2416855954.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_einspielen = df_ohne_dubletten.append(df_dubl_einspielen)                    # die ausgewählten Dubletten und alle Nicht-Dubletten werden in ein Datenframe zusammengeführt


<hr>

## 3. Überblick zu den Daten und Trennen in "in Aleph vorhanden" und "neu"

In [28]:
m = df_einspielen.shape[0]
n = df_einspielen["ids"].count()
print("Zum Einspielen:", m)
print("Davon in Aleph:", n)
print("Neue Titel:", m-n)

Zum Einspielen: 54468
Davon in Aleph: 27581
Neue Titel: 26887


In [29]:
df_in_aleph = df_einspielen.dropna(subset=['ids'])
df_in_aleph.reset_index(inplace=True)
df_in_aleph.shape[0]

# zur Prüfung am GES und Ebx Bestand geprüft werden

27581

In [30]:
df_ganz_neu = df_einspielen[df_einspielen['ids'].isnull()]

df_ganz_neu.reset_index(inplace=True)
df_ganz_neu.shape[0]

# diese müssen ebenfalls am Ges und Ebx Bestand geprüft werden, durch Trennung der Sets schnellere Bearbeitung

26887

<hr>

## 2. Bestandsabfragen



*URLs für die Abfrage über den X-Server unseres Bibliothekssystems werden erzeugt und über die ISBN eine Abfrage auf Bestand gemacht. Die Abfrage funktioniert nur für zugelassene IPs (darum oben die Prüfung).  
Für die Abfrage in unseren Bestand ist die ISBN sehr gut, da in den Titeldaten alle im Buch befindlichen ISBNs - auch die anderer Ausgabeformen - mit übernommen sind. Beim MPG-Ebooks Katalog handelt sich um Daten von Verlagen, die sich in ihrer Qualität und Informationsumfang sehr unterscheiden. Hier wird noch zu prüfen sein, inwieweit ein anderer Abfragemechansimus gewählt werden muss.*  

### 1. Ganz neue Titel  

= df_ganz_neu


In [31]:
df_ganz_neu["url_ges"] = df_ganz_neu["isbn_ean"].apply(lambda x: f"ges_link{x}".replace('ges_link','http://aleph.mpg.de/X?op=find&base=ges01&request=IBS='))

#Es funktionierte nicht, dass die URLs an die vorhandenen ISBNs einfach so angefügt werden, darum der Workaround mit einem Platzhalter, der sich dann über replace vom richtigen Link überschreiben ließ.

#### URLs für Ebooks-Katalog erzeugen 

In [32]:
df_ganz_neu["url_ebx"] = df_ganz_neu["isbn_ean"].apply(lambda x: f"ebx_link{x}".replace('ebx_link', 'http://aleph.mpg.de/X?op=find&base=ebx01&request=IBN='))


#### Abfragen beim Server

##### Zunächst für die Daten des MPIfG 

*Vorgehensweise: Abfrage und Sammeln der Antworten in einer Datei, diese Antworten werden dann in Ausdrücke "übersetzt" - "vorhanden" und "neu" und diese Daten in eine Spalte ins Dataframe zur weiteren Auswertung übertragen.   
Schwierigkeit hier war, die Sammlung der Antworten zu den einzelnen Titeln, um sie in das Datenframe einzuspielen. Der störende XML-Header der Antworten wird erst gar nicht in die Datei geschrieben.   *   

In [33]:
with open('./input/server_responses', 'w') as fn:  
    for url in df_ganz_neu["url_ges"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses', 'r') as f:
    with open('./input/server_responses_transfered', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf = pd.read_fwf('./input/server_responses_transfered', names=["Abfrage_ges"])
df_result = df_ganz_neu.join(df_fwf)                    #df_result = pd.concat([df_ganz_neu, df_fwf], axis=1), funktioniert nicht mehr                  

    Randnotiz: 
    Bei 2400 Titels brauchte der Abgleich ca 350 Sekunden

In [34]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind, wird mittelfristig rausfallen
x = df_ganz_neu.shape[0]
y = df_fwf.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

Anzahl der eingelesenen Datensätze: 26887 
Anzahl der Antworten vom Server:    26887


##### Datenabgleich mit dem Bestand des MPG Ebooks-Katalog

*Vorgehensweise analog Bestandsabfrage MPI.*

In [35]:
with open('./input/server_responses_ebx', 'w') as fn:  
    for url in df_result["url_ebx"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses_ebx', 'r') as f:
    with open('./input/server_responses_transfered_ebx', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf_ebx = pd.read_fwf('./input/server_responses_transfered_ebx', names=["Abfrage_ebx"])
df_result_neu = df_result.join(df_fwf_ebx)           #df_result_neu = pd.concat([df_result, df_fwf_ebx], axis=1)

In [36]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind
x = df_result_neu.shape[0]
y = df_fwf_ebx.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

Anzahl der eingelesenen Datensätze: 26887 
Anzahl der Antworten vom Server:    26887


<hr>

### 2. Titel in Aleph vorhanden

= df_in_aleph

*Hier reicht Abgleich mit Ebooks, das erworbene Bücher manuell gelöscht werden.*

In [37]:
df_in_aleph["url_ges"] = df_in_aleph["isbn_ean"].apply(lambda x: f"ges_link{x}".replace('ges_link', 'http://aleph.mpg.de/X?op=find&base=ges01&request=IBS='))

df_in_aleph["url_ebx"] = df_in_aleph["isbn_ean"].apply(lambda x: f"ebx_link{x}".replace('ebx_link', 'http://aleph.mpg.de/X?op=find&base=ebx01&request=IBN='))


#### Abfragen beim Server


In [38]:
with open('./input/server_responses2', 'w') as fn:  
    for url in df_in_aleph["url_ges"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses2', 'r') as f:
    with open('./input/server_responses_transfered2', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf_ges = pd.read_fwf('./input/server_responses_transfered2', names=["Abfrage_ges"])
df_result2 = df_in_aleph.join(df_fwf_ges)               #df_result2 = pd.concat([df_in_aleph, df_fwf_ges], axis=1)             

In [39]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind, wird mittelfristig rausfallen
x = df_in_aleph.shape[0]
y = df_fwf_ges.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

Anzahl der eingelesenen Datensätze: 27581 
Anzahl der Antworten vom Server:    27581


In [40]:
df_result2['Abfrage_ges']

0        neu
1        neu
2        neu
3        neu
4        neu
        ... 
27576    neu
27577    neu
27578    neu
27579    neu
27580    neu
Name: Abfrage_ges, Length: 27581, dtype: object

In [41]:
with open('./input/server_responses_ebx2', 'w') as fn:  
    for url in df_result2["url_ebx"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses_ebx2', 'r') as f:
    with open('./input/server_responses_transfered_ebx2', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf_ebx2 = pd.read_fwf('./input/server_responses_transfered_ebx2', names=["Abfrage_ebx"])
df_result_in_aleph = df_result2.join(df_fwf_ebx2)           #df_result_in_aleph = pd.concat([df_result2, df_fwf_ebx2], axis=1)

In [42]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind
x = df_result_in_aleph.shape[0]
y = df_fwf_ebx2.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

Anzahl der eingelesenen Datensätze: 27581 
Anzahl der Antworten vom Server:    27581


<hr>

## 3. Exportvorbereitungen 


### 1. Neue Titel

In [43]:
df_neu_nicht_einspielen =  df_result_neu.drop(df_result_neu[(df_result_neu["Abfrage_ebx"]== 'neu') & (df_result_neu["Abfrage_ges"] == 'neu')].index)          
# alle Titel rausholen, die in einer der beiden Datenbanken vorhanden waren, diese werden mit den vorhandenen aus Update unten in Excel geschrieben


#### Jetzt Extraktion der Titel zum Einspielen:

In [44]:
df_neu_aleph_einspielen = df_result_neu.loc[((df_result_neu["Abfrage_ebx"]== 'neu') & (df_result_neu["Abfrage_ges"] == 'neu'))]   #das sind die komplett neuen Titel

### 2. In Aleph vorhandene Titel

*Hier ist der Fall: was im Ebooks-Katalog vorhanden ist, muss gelöscht werden*

In [45]:
df_in_aleph_nicht_einspielen = df_result_in_aleph.drop(df_result_in_aleph[(df_result_in_aleph["Abfrage_ebx"] == 'neu') & (df_result_in_aleph["Abfrage_ges"] == 'neu')].index) 
df_in_aleph_nicht_einspielen

,level_0,index,object_id,isbn_ean,title,subtitle,contributor,publisher,series,thesis,...,title_sep,subtitle_sep,subtitle_comparison,subtitle_comparison2,subtitle_all,short_title,url_ges,url_ebx,Abfrage_ges,Abfrage_ebx
71,340,559,2590888,9781394177417,Canoeing Under Elephants: Leading Organisation...,Driving Sustainability in Business,S McLachlan,Wiley,,,...,Canoeing Under Elephants,Leading Organisations Fr om ESG Compliance to...,Driving Sustainability in Business,,Driving Sustainability in Business,Canoeing Under Elephants Driving Sustainabilit...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,vorhanden
87,360,582,2612729,9781119880936,"Financial Valuation, + Website",Applications and Models,James R. Hitchner,Wiley,Wiley Finance,,...,"Financial Valuation, + Website",,Applications and Models,,Applications and Models,"Financial Valuation, + Website Applications an...",http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,vorhanden
91,364,587,2619087,9781119880974,Financial Valuation Workbook,Step-by-Step Exercises and Tests to Help You M...,James R. Hitchner,Wiley,Wiley Finance,,...,Financial Valuation Workbook,,Step-by-Step Exercises and Tests to Help You M...,,Step-by-Step Exercises and Tests to Help You M...,Financial Valuation Workbook Step-by-Step Exer...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,vorhanden
112,399,655,2621617,9783848775088,Sozialwirtschaft,"Handbuch für Wissenschaft, Studium und Praxis",Klaus Grunwald,Nomos,NomosHandbuch,,...,Sozialwirtschaft,,"Handbuch für Wissenschaft, Studium und Praxis",,"Handbuch für Wissenschaft, Studium und Praxis","Sozialwirtschaft Handbuch für Wissenschaft, St...",http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,vorhanden
152,453,768,2628406,9783110437577,Handbuch globale Handelsräume und Handelsrouten,Von der Antike bis zur Gegenwart,Mark Häberlein,De Gruyter Oldenbourg,De Gruyter Reference,,...,Handbuch globale Handelsräume und Handelsrouten,,Von der Antike bis zur Gegenwart,,Von der Antike bis zur Gegenwart,Handbuch globale Handelsräume und Handelsroute...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,vorhanden
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26343,51722,3485,153617203,9780367456412,Digital Analytics for Marketing,,A. Karim Feroz;Gohar F. Khan;Marshall Sponder,Routledge,Mastering Business Analytics,,...,Digital Analytics for Marketing,,,,,Digital Analytics for Marketing / A. Karim Fe...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,vorhanden
26657,52399,9393,159676760,9781032062006,Contemporary Issues in Marketing and Consumer ...,,Elizabeth Parsons;Pauline Maclaran;Andreas Cha...,Routledge,,,...,Contemporary Issues in Marketing and Consumer ...,,,,,Contemporary Issues in Marketing and Consumer ...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,vorhanden
26677,52437,40934,167759716,9781032388342,Consumer Behaviour and Analytics,,Andrew Smith,Routledge,Mastering Business Analytics,,...,Consumer Behaviour and Analytics,,,,,Consumer Behaviour and Analytics / Andrew Smith,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,vorhanden
26954,53085,42599,168045256,9781032514147,Business Model Innovation,How it really works,Staffan Heden,Routledge,,,...,Business Model Innovation,,How it really works,,How it really works,Business Model Innovation How it really works ...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,vorhanden


In [46]:
# Erzeugen der Datei zum Löschen der Titel

with open("./output/ges02_weg", "a", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_in_aleph_nicht_einspielen.index:
        fa.write(df_in_aleph_nicht_einspielen["ids"][i]+'GES02'+'\n')

Für die Statistik und Kontrolle zusammenführen aller Titel, die nicht eingespielt werden und Ausgabe in einer CSV

In [47]:
df_gesamt_nicht_einspielen = df_in_aleph_nicht_einspielen.append(df_neu_nicht_einspielen)   

C:\Users\cm\AppData\Local\Temp\ipykernel_3328\2360574726.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_gesamt_nicht_einspielen = df_in_aleph_nicht_einspielen.append(df_neu_nicht_einspielen)


In [48]:
date = time.strftime("%Y_%m_%d")                                              # Zeit erfassen für Dateibenennung

df_gesamt_nicht_einspielen["object_id"] = df_gesamt_nicht_einspielen.object_id.astype(str)  # wandelt die spalte von Int64 zu Object um, so dass es in Excel korrekt eingelesen wird
df_gesamt_nicht_einspielen["isbn_ean"] = df_gesamt_nicht_einspielen.isbn_ean.astype(str)
df_gesamt_nicht_einspielen = df_gesamt_nicht_einspielen.drop(columns=["url_ebx", "url_ges", "cover", "title_sep", "subtitle_comparison", "subtitle_comparison2", "subtitle_all", "subtitle_sep"]) # unnötige Spalten entfernen

df_neu_nicht_einspielen.to_csv('./output/Vorhandene_Titel_'+date+'.csv')   

#### Jetzt Extraktion der Titel zum Updaten:

In [49]:
df_in_aleph_update = df_result_in_aleph.loc[(df_result_in_aleph["Abfrage_ebx"]== 'neu') & (df_result_in_aleph["Abfrage_ges"] == 'neu')] # prüfen, ob es wirklich ein Datenupdate gab, sonst nicht neu einspielen??

<hr>

#### Für die Logdatei Ermittlung verschiedener Zahlen und hier zur direkten Ansicht ausgegeben

In [50]:
#Kontrollmechanismus, ob für alle Titel auch Treffer da sind
x = df.shape[0]
a = df_ohne_dubletten.shape[0]
c = df_dubletten.shape[0]
b = df_dubl_einspielen.shape[0] #Auswahl der neuen Treffer
m = df_in_aleph_nicht_einspielen.shape[0]
n = df_ganz_neu.shape[0]
h = df_neu_aleph_einspielen.shape[0]
j = df_neu_nicht_einspielen.shape[0]
o = df_in_aleph_update.shape[0]
g = df_in_aleph.shape[0]
k = df_alephIDs.shape[0]
z = df_neu_aleph_einspielen.shape[0]       #neu ermitteln aus neuen
l = df_aleph_loeschen.shape[0]
y = df_dubl_nicht_einspielen.shape[0]


print('Kleine Statistik:\n=====================================',
    '\nGelieferte Datensätze:             ', x,
    '\n--------------------------------------------', 
    '\nSätze ohne Dubletten               ', a,
    '\n   Dubletten:              ', c,
    '\n   Auswahl zum Einspielen  ', b,
    '\n   in Aleph löschen        ', y,
    '\nZu prüfende Datensätze:            ', a+b,
    '\n--------------------------------------------', 
    '\nPrüfung Titel in Aleph:            ', g,
    '\n   Davon als Update        ', o,
    '\n   Davon in Aleph löschen  ', m ,    
    '\nPrüfung neue Titel    :            ', n,
    '\n   Davon ganz neu          ', h,
    '\n   Davon bereits Bestand   ', j,      
    '\n--------------------------------------------', 
    '\nTitel, die in Aleph verarbeitet werden: ', o+h,
    '\n--------------------------------------------', 
    '\nDatensätze GES02 vor Einspielen:   ', k,
    '\nNicht mehr im Export               -', l,
    '\nTitel in Aleph löschen, da Bestand -', m,
    '\nZu löschen, da Dublette            -', y,
    '\nNeue Titel für Aleph               +', h,
    '\n--------------------------------------------', 
    '\nIn Aleph nach Einspielen:          ', k-l-m-y+h,)

#hier entsprechende Einträge für die Log-Datei

with open ('./log/pda_import_log.txt', 'a') as log:
    log.write("\nGelieferte Datensätze:             " + str(x))
    log.write("\n--------------------------------------------\n")
    log.write("\nSätze ohne Dubletten               " + str(a))
    log.write("\n   Dubletten:              " + str(c))
    log.write("\n   Davon in Aleph löeschen " + str(y))
    log.write("\n   Auswahl zum Einspielen  " + str(b))
    log.write("\nZu prüfende Datensätze:            " + str(a+b))
    log.write("\n--------------------------------------------\n")
    log.write("\nPrüfung Titel in Aleph:            " + str(g))
    log.write("\n   Davon als Update        " + str(o))
    log.write("\n   Davon in Aleph löschen  " + str(m) + "    \n")   
    log.write("\nPrüfung neue Titel    :            " + str(n))
    log.write("\n   Davon ganz neu          " + str(h))
    log.write("\n   Davon bereits Bestand   " + str(j) + "    \n")     
    log.write("\n--------------------------------------------\n")
    log.write("\nTitel, die in Aleph verarbeitet werden: " + str(o+h))
    log.write("\n--------------------------------------------\n")
    log.write("\nDatensätze GES02 vor Einspielen:   " + str(k))
    log.write("\nNicht mehr im Export               -" + str(l))
    log.write("\nZu löschender dubletter Titel      -" + str(y))
    log.write("\nTitel in Aleph löschen, da Bestand -" + str(m))
    log.write("\nNeue Titel für Aleph               +" + str(h))
    log.write("\n--------------------------------------------\n")
    log.write("\nIn Aleph nach Einspielen:          " + str(k-l-m-y+h))
    log.write("\n--------------------------------------------\n")

Kleine Statistik:
Gelieferte Datensätze:              66994 
-------------------------------------------- 
Sätze ohne Dubletten                42108 
   Dubletten:               24886 
   Auswahl zum Einspielen   12360 
   in Aleph löschen         0 
Zu prüfende Datensätze:             54468 
-------------------------------------------- 
Prüfung Titel in Aleph:             27581 
   Davon als Update         27243 
   Davon in Aleph löschen   338 
Prüfung neue Titel    :             26887 
   Davon ganz neu           23306 
   Davon bereits Bestand    3581 
-------------------------------------------- 
Titel, die in Aleph verarbeitet werden:  50549 
-------------------------------------------- 
Datensätze GES02 vor Einspielen:    29972 
Nicht mehr im Export               - 210 
Titel in Aleph löschen, da Bestand - 338 
Zu löschen, da Dublette            - 0 
Neue Titel für Aleph               + 23306 
-------------------------------------------- 
In Aleph nach Einspielen:           5273

<hr>

## 4. Exportdateien Aufbereiten

#### Zielformat für das Einspielen in Aleph:

    000000001 LDR   L -----nM2.01200024------h              
    000000001 020   L $$a (object_id))
    000000001 030   L $$aaz||rrrza||||
    000000001 051   L $$am|||||||
    000000001 070   L $$aSchweitzer
    000000001 077   L $$aMonographie
    000000001 078   L $$aSchweitzer
    000000001 082   L $$azum Bestellen
    000000001 100   L $$a (contributor_1)
    000000001 104   L $$a (contributor_2)
    000000001 108   L $$a (contributor_3)
    000000001 331   L $$a (title_sep)
    000000001 335   L $$a (subtitle_all)
    000000001 403   L $$a (edition_number / edition_text)  #noch prüfen, was besser zu verwenden ist 
    000000001 419   L $$b (publisher) $$a (date_combined)
    000000001 433   L $$a (pages)
    000000001 451   L $$a (series)
    000000001 520   L $$a (thesis)
    000000001 540   L $$a (isbn_ean)
    000000001 656   L $$a (cover)
    000000001 750   L $$a (description)
    000000001 655   L $$zOrder me$$umailto:bib@mpifg.de?subject=Bestellwunsch        
    
Anmerkung zum Feld 655: die URL wird NACH dem Einspielen in Aleph mit der Datensatz-ID angereichert (siehe Juypter-Notebook "Link-Anreicherung"), um einen klaren Bestellink für den Kaufvorschlag zu haben

*Hierfür werden immer die Feldbenennung und bestimmte Codierungen VOR den Inhalt - in Klammern de Bezeichnung der entsprechenden Spalte - gesetzt, bzw. erfoderliche Felder komplett neu hinzugefügt.   
Am Anfang jeder Zeile braucht Aleph eine 9-Stellige eindeutige Zahl pro Titel.*   

*Manchmal ließ sich der Inhalt einer Spalte direkt in die Datei schreiben, manchmal musst die Spalte zuvor über apply aufbereitet werden.* 

## 1. Aufbereiten der neuen Titel

In [51]:
df_aleph_einspielen = df_neu_aleph_einspielen           #zur vereinfachten Wiederverwertung des alten Codes

In [52]:
df_aleph_einspielen["020"] = df_aleph_einspielen["object_id"].apply(lambda x: f"020   L $$a{x}") 
del df_aleph_einspielen["object_id"]                                                                      #um das df nicht unnötig anwachsen zu lassen, jeweils alte Spalte löschen

#### Besondere Aufbereitung der Personendaten

*Da bis zu 3 Personen in einer Spalte zu finden sind, werden diese im Discovery nicht getrennt suchbar, darum werden sie gesplittet. Für die Dublettenkontrolle hat sich das als irrelevant erwiesen, darum erfolgt dieser Schritt erst hier.*

In [53]:
person = df_aleph_einspielen["contributor"].str.split(';', expand=True)                         #Für saubere Daten, die Autorenangabe splitten und in getrennte Felder schreiben

df_aleph_einspielen["contributor_1"]= person[0]
df_aleph_einspielen["contributor_2"]= person[1]
df_aleph_einspielen["contributor_3"]= person[2]

df_aleph_einspielen["contributor_1"]= df_aleph_einspielen["contributor_1"].replace(np.nan, '', regex=True)
df_aleph_einspielen["contributor_2"]= df_aleph_einspielen["contributor_2"].replace(np.nan, '', regex=True)
df_aleph_einspielen["contributor_3"]= df_aleph_einspielen["contributor_3"].replace(np.nan, '', regex=True)


#### Besondere Aufbereitung des Erscheinungsdatum und Erscheinungsjahres

*In der Auswahl unserer Titel befinden sich auch im Erscheinen befindliche Titel der kommenden Monate. Diese Information möchten wir gerne im Discovery sichtbar machen. Hierfür bleibt uns nur Aleph-Feld 419c, das dem Erscheinungsjahr vorbehalten ist.   
Wunsch ist es: Wenn des Erscheinungsdatum weiter als 10 Tage weg vom heutigen Datum ist, soll das komplette Datum angezeigt werden, ansonsten nur das Erscheinungsjahr.*

Zur Umsetzung muss die Spalte "publication_date" in ein Datum verwandelt werden und nach den genannten Kriterien unterschiedlich angezeigt

In [54]:
df_aleph_einspielen["publ_data_repl"]= df_aleph_einspielen["publication_date"].astype(str).str.replace('00','01') # korrektur komischer Daten
#df_aleph_einspielen["publ_data_repl"]

In [55]:
df_aleph_einspielen["publ_date_date"] = pd.to_datetime(df_aleph_einspielen["publ_data_repl"].astype(str), format='%Y%m%d', errors='coerce') #Umwandlung in Datum (Zahl geht nicht)
#df_aleph_einspielen["publ_date_date"]

In [59]:
today = pd.Timestamp(datetime.today().date())  # Heute 

def transform_date(date):
    if date > today:
        return date.strftime('%Y-%m-%d')
    else:
        return date.year

# Apply the function to create the new column
df_aleph_einspielen['date_combined'] = df_aleph_einspielen['publ_date_date'].apply(transform_date)
#df_aleph_einspielen['date_combined']

In [60]:
df_aleph_einspielen['isbn_ean'] = df_aleph_einspielen['isbn_ean'].astype(np.int64)   #da die Zahl als Object genommen wurde, bekam sie ein .0 angehängt, das ist durch umwandeln in Zahl weg

df_aleph_einspielen['isbn_ean']

0        9783406582387
1        9783406753732
2        9783406767890
3        9783848728732
4        9783825287887
             ...      
26882    9781350329447
26883    9783030875183
26884    9781350242401
26885    9781009523103
26886    9781961856783
Name: isbn_ean, Length: 23306, dtype: int64

In [61]:
df_aleph_einspielen["419b"] = df_aleph_einspielen["publisher"].apply(lambda x: f"419   L $$b{x}") 
df_aleph_einspielen["419c"] = df_aleph_einspielen["date_combined"].apply(lambda x: f"$$c{x}")                  

df_aleph_einspielen["419"] = df_aleph_einspielen["419b"]+df_aleph_einspielen["419c"]                           #Für die Korrekte Eingabe brauche ich Verlag und Jahr in einer Spalte

In [62]:
df_aleph_einspielen["403"] = df_aleph_einspielen["edition_text"].apply(lambda x: f"403   L $$a{x}") 
df_aleph_einspielen["433"] = df_aleph_einspielen["pages"].apply(lambda x: f"433   L $$b{x}")
df_aleph_einspielen["451"] = df_aleph_einspielen["series"].apply(lambda x: f"451   L $$b{x}") 
df_aleph_einspielen["520"] = df_aleph_einspielen["thesis"].apply(lambda x: f"520   L $$a{x}") 
df_aleph_einspielen["540"] = df_aleph_einspielen["isbn_ean"].apply(lambda x: f"540   L $$a{x}") 
df_aleph_einspielen["656"] = df_aleph_einspielen["cover"].apply(lambda x: f"656   L $$u{x}") 
df_aleph_einspielen["750"] = df_aleph_einspielen["description"].apply(lambda x: f"750   L $$a{x}") 

#### Für das Durchzählen der Titel braucht es eine neue Spalte

In [63]:
# hier entsteht eine neue Spalte mit Zahlen ab 1 durchgehend gezählt, die für den korrekten Import der Daten in Aleph nötig ist
x = df_aleph_einspielen.shape[0]   
df_aleph_einspielen["id"] = range(1,x+1)                                                       #Notwendig ist die Zählung ab 1, da Aleph sonst nicht korrekt einließt
df_aleph_einspielen["id"] = df_aleph_einspielen["id"].apply(lambda x: f"{x:09d}")              #Die Zahl muss 9-Stellig aufgefüllt werden



#### Vorbereitungen abgeschlossen, jetzt das Schreiben der Datei im Aleph-Sequential-Format:

In [64]:
with open("./output/ges02_neu", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_aleph_einspielen.index:
        fa.write(df_aleph_einspielen["id"][i]+' LDR   L -----nM2.01200024------h'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["020"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 030   L $$aaz||rrrza||||'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 051   L $$am|||||||m|||||||'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 077   L $$aMonographie'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 078   L $$aSchweitzer'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 082   L $$azum Bestellen'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 100   L $$a'+df_aleph_einspielen["contributor_1"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 104   L $$a'+df_aleph_einspielen["contributor_2"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 108   L $$a'+df_aleph_einspielen["contributor_3"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 331   L $$a'+df_aleph_einspielen["title_sep"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 335   L $$a'+df_aleph_einspielen["subtitle_all"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["403"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["419"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["433"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["451"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["520"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["540"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 655   L $$zOrder me$$umailto:bib@mpifg.de?subject=Bestellwunsch'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["656"][i]+'$$3Cover\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["750"][i]+'\n')

<hr>

### 2. Daten für Update

*es wird anhand der Spalte "last_modified" geprüft, ob die Titel seit dem letzten Einspielen, bzw. seit 30 Tagen ein Update erfahren haben*


Dies mache ich, um die Menge zum Updaten zu reduzieren. Es werden nur die rausgezogen, die tatsächlich ein neueres Aktualisierungsdatum haben.

In [68]:
one_month_ago = datetime.now() - timedelta(days=30)

In [69]:
one_month_ago = pd.Timestamp(one_month_ago)
one_month_ago

Timestamp('2025-02-23 14:37:45.627436')

In [70]:
df_in_aleph_update['last_modified'] = pd.to_datetime(df_in_aleph_update.last_modified)          #Umwandlung, da Spalteninhalt object ist
df_in_aleph_update['last_modified']

0       2025-03-23 21:03:31
1       2025-02-07 06:11:51
2       2025-03-17 20:12:18
3       2025-02-20 20:40:50
4       2025-02-18 06:40:09
                ...        
27576   2025-02-28 17:30:14
27577   2025-02-24 17:38:08
27578   2025-02-12 02:13:37
27579   2025-03-13 18:23:26
27580   2025-03-11 18:23:53
Name: last_modified, Length: 27243, dtype: datetime64[ns]

In [71]:
df_in_aleph_update_true = df_in_aleph_update[df_in_aleph_update['last_modified'].dt.to_period('M') == one_month_ago.to_period('M')]

#df_in_aleph_update_true

#### Erstellen der Update-Export-Datei

In [72]:
df_in_aleph_update_true["020"] = df_in_aleph_update_true["object_id"].apply(lambda x: f"020   L $$a{x}") 
del df_in_aleph_update_true["object_id"]                                                                      #um das df nicht unnötig anwachsen zu lassen, jeweils alte Spalte löschen

In [73]:
#Aufbereitung Personendaten

person = df_in_aleph_update_true["contributor"].str.split(';', expand=True)                         #Für saubere Daten, die Autorenangabe splitten und in getrennte Felder schreiben

df_in_aleph_update_true["contributor_1"]= person[0]
df_in_aleph_update_true["contributor_2"]= person[1]
df_in_aleph_update_true["contributor_3"]= person[2]

df_in_aleph_update_true["contributor_1"]= df_in_aleph_update_true["contributor_1"].replace(np.nan, '', regex=True)
df_in_aleph_update_true["contributor_2"]= df_in_aleph_update_true["contributor_2"].replace(np.nan, '', regex=True)
df_in_aleph_update_true["contributor_3"]= df_in_aleph_update_true["contributor_3"].replace(np.nan, '', regex=True)


### Neue Schritte zur Aufbereitung des Erscheinungsdatums, da der Code von oben hier nicht ging - warum auch immer

In [74]:
df_in_aleph_update_true["publ_data_repl"]= df_in_aleph_update_true["publication_date"].astype(str).str.replace('00','01')
df_in_aleph_update_true["publ_date_date"] = pd.to_datetime(df_in_aleph_update_true["publ_data_repl"].astype(str), format='%Y%m%d', errors='coerce')

# Apply the function von oben to create the new column
df_in_aleph_update_true['date_combined'] = df_in_aleph_update_true['publ_date_date'].apply(transform_date)
df_in_aleph_update_true['date_combined']

1        2025-05-01
3        2025-08-31
4        2025-05-12
5        2025-06-30
7        2025-12-31
            ...    
27574          2023
27575          2024
27576          2024
27577          2024
27578          2023
Name: date_combined, Length: 10652, dtype: object

In [75]:
df_in_aleph_update_true['date_combined'].value_counts()

2024          5783
2023          4763
2025            33
2021            12
2025-12-31      10
2025-12-01       7
2025-05-01       4
2025-06-01       4
2025-05-15       2
2025-04-10       2
2025-05-22       1
2025-11-20       1
2026-06-30       1
2022             1
2025-07-08       1
2025-09-01       1
2025-05-06       1
2025-04-07       1
2025-12-03       1
2025-08-06       1
2025-04-04       1
2026-04-30       1
2025-10-10       1
2025-04-13       1
2025-11-01       1
2025-06-16       1
2025-08-01       1
2025-08-31       1
2025-06-11       1
2025-12-11       1
2026-01-01       1
2012             1
2025-07-01       1
2026-06-01       1
2025-06-30       1
2025-05-12       1
2025-08-14       1
Name: date_combined, dtype: int64

In [76]:
df_in_aleph_update_true['isbn_ean'] = df_in_aleph_update_true['isbn_ean'].astype(np.int64)   #da die Zahl als Object genommen wurde, bekam sie ein .0 angehängt, das ist durch umwandeln in Zahl weg

df_in_aleph_update_true['isbn_ean']

1        9783848768073
3        9783406701948
4        9783825254476
5        9783406645877
7        9783825250614
             ...      
27574    9781032466965
27575    9783110710045
27576    9781032411460
27577    9781032520759
27578    9783030962708
Name: isbn_ean, Length: 10652, dtype: int64

In [77]:
# Bearbeitung der Felder

df_in_aleph_update_true["419b"] = df_in_aleph_update_true["publisher"].apply(lambda x: f"419   L $$b{x}") 
df_in_aleph_update_true["419c"] = df_in_aleph_update_true["date_combined"].apply(lambda x: f"$$c{x}")                  

df_in_aleph_update_true["419"] = df_in_aleph_update_true["419b"]+df_in_aleph_update_true["419c"]                           #Für die Korrekte Eingabe brauche ich Verlag und Jahr in einer Spalte

df_in_aleph_update_true["403"] = df_in_aleph_update_true["edition_text"].apply(lambda x: f"403   L $$a{x}") 
df_in_aleph_update_true["433"] = df_in_aleph_update_true["pages"].apply(lambda x: f"433   L $$b{x}")
df_in_aleph_update_true["451"] = df_in_aleph_update_true["series"].apply(lambda x: f"451   L $$b{x}") 
df_in_aleph_update_true["520"] = df_in_aleph_update_true["thesis"].apply(lambda x: f"520   L $$a{x}") 
df_in_aleph_update_true["540"] = df_in_aleph_update_true["isbn_ean"].apply(lambda x: f"540   L $$a{x}") 
df_in_aleph_update_true["656"] = df_in_aleph_update_true["cover"].apply(lambda x: f"656   L $$u{x}") 
df_in_aleph_update_true["750"] = df_in_aleph_update_true["description"].apply(lambda x: f"750   L $$a{x}") 

In [78]:
# Schreiben der Ausgabedatei, hier kleine Unterschiede zu den neuen Titeln. Vorhandene ids und bestimmte Felder können nicht verändert sein, brauchen also nicht übernommen zu werden.

with open("./output/ges02_update", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_in_aleph_update_true.index:
        fa.write(df_in_aleph_update_true["ids"][i]+' 100   L $$a'+df_in_aleph_update_true["contributor_1"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 104   L $$a'+df_in_aleph_update_true["contributor_2"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 108   L $$a'+df_in_aleph_update_true["contributor_3"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 331   L $$a'+df_in_aleph_update_true["title_sep"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 335   L $$a'+df_in_aleph_update_true["subtitle_all"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["403"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["419"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["433"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["451"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["520"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["540"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["656"][i]+'$$3Cover\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["750"][i]+'\n')

<hr>

## Abschließende Dinge

In [79]:
#Abschließender Eintrag in Log-Datei
endtime = time.strftime('%H:%M')

with open ('./log/pda_import_log.txt', 'a') as log:                                                  # Da diese Log-Datei nicht unmittelbar gebraucht wird, hier fortlaufendes Schreiben in eine Datei
    log.write('\n                                     beendet ')
    log.write(endtime)
    log.write("\n============================================================\n\n")

Kopien bestimmter Daten zur Einsicht bzw. für Prüfzwecke

In [80]:
df_aleph_einspielen.to_csv('./output/Eingespielte_Titel_'+date+'.csv') 
df_in_aleph_update_true.to_csv('./output/Update_Titel_'+date+'.csv')

df_in_aleph_nicht_einspielen.to_csv('./output/Aleph_loeschen_ebx_vorh_'+date+'.csv')

<hr>

Folgende Schritte müssen mit den Dateien ausgeführt werden: 


1. Einspielen der Datei pda_ges01 als neue Titel in Aleph, hier dann auch Export der urls und Anreicherung mit der Aleph-ID mittels "mailto_link_skript.ipynb"
2. Einspielen der DAtei pda_update als "Änderungen bestehender Datensätze in Aleph"
3. Einspielen und löschen der Titel die in ges02_loeschen_1 und ges02_loeschen_2 vorhanden sind